<a href="https://colab.research.google.com/github/Filipe-Ruas/Cerrado/blob/main/App_supermercado_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template_string, request, redirect, url_for, flash
from waitress import serve
import os

app = Flask(__name__)
app.secret_key = os.environ.get("SECRET_KEY", "supersecretkey")

# Placeholder for items
items = []

html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Lista de Supermercado</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f8ff;
            margin: 0;
            padding: 0;
        }
        .container {
            max-width: 600px;
            margin: 40px auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        h1 {
            color: #2a9d8f;
            text-align: center;
            font-size: 2.4em;
        }
        .input-group {
            display: flex;
            margin-bottom: 20px;
        }
        .input-group input {
            flex: 1;
            padding: 12px;
            border: 1px solid #ccc;
            border-radius: 6px 0 0 6px;
            font-size: 1.1em;
        }
        .input-group button {
            padding: 12px;
            background-color: #2a9d8f;
            color: white;
            border: none;
            border-radius: 0 6px 6px 0;
            cursor: pointer;
            font-size: 1.1em;
        }
        .input-group button:hover {
            background-color: #21867a;
        }
        ul {
            list-style-type: none;
            padding: 0;
        }
        li {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 12px;
            border: 1px solid #ddd;
            margin-bottom: 10px;
            border-radius: 6px;
            background-color: #f9f9f9;
        }
        li:hover {
            background-color: #e9f5f3;
        }
        .delete {
            background: none;
            border: none;
            color: #e63946;
            font-size: 1em;
            cursor: pointer;
        }
        .delete:hover {
            color: #d62828;
        }
        .flash-message {
            background-color: #ffebee;
            color: #b71c1c;
            padding: 10px;
            margin-bottom: 20px;
            border: 1px solid #b71c1c;
            border-radius: 6px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Lista de Supermercado</h1>
        {% with messages = get_flashed_messages() %}
        {% if messages %}
        <div class="flash-message">
            {{ messages[0] }}
        </div>
        {% endif %}
        {% endwith %}
        <form action="/add" method="POST" class="input-group">
            <input type="text" name="item" placeholder="Adicionar Item" required>
            <button type="submit">Adicionar</button>
        </form>
        <ul>
            {% for item in items %}
            <li>
                {{ item }}
                <form action="/delete/{{ loop.index0 }}" method="POST" style="margin: 0;">
                    <button type="submit" class="delete">Remover</button>
                </form>
            </li>
            {% endfor %}
        </ul>
    </div>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(html_template, items=items)

@app.route('/add', methods=['POST'])
def add_item():
    item = request.form['item'].strip()
    if item:
        items.append(item)
        flash("Item adicionado com sucesso!", 'success')
    else:
        flash("Por favor, insira um item válido.", 'error')
    return redirect(url_for('index'))

@app.route('/delete/<int:item_index>', methods=['POST'])
def delete_item(item_index):
    if 0 <= item_index < len(items):
        items.pop(item_index)
        flash("Item removido com sucesso!", 'success')
    else:
        flash("Não foi possível remover o item.", 'error')
    return redirect(url_for('index'))

if __name__ == '__main__':
    port = int(os.environ.get("PORT", 5000))
    debug_mode = os.environ.get("FLASK_DEBUG", "false").lower() == "true"
    if debug_mode:
        app.run(host='0.0.0.0', port=port, debug=True)
    else:
        serve(app, host='0.0.0.0', port=port)
